In [23]:
from skimage.io import imread, imshow
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from transformers import AutoImageProcessor, SegformerForSemanticSegmentation, SegformerModel, SegformerConfig, AutoFeatureExtractor

image_processor = AutoImageProcessor.from_pretrained("nvidia/segformer-b0-finetuned-ade-512-512")

id2label = {
    '0': 'flood'
}

label2id = {v:k for k, v in id2label.items()}
model = SegformerForSemanticSegmentation.from_pretrained("nvidia/segformer-b0-finetuned-ade-512-512", ignore_mismatched_sizes=True,
                                                        num_labels=len(id2label), id2label=id2label, label2id=label2id,
                                                        reshape_last_stage=True)

from etci_dataset import ETCIDataset, ETCIDataModule
datamodule=ETCIDataModule('../data/', batch_size=8, num_workers=1,
                              debug=True, transforms=False)

datamodule.setup()

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
/home/genrev_kiel_hizon/sar_env_2/lib/python3.9/site-packages/transformers/models/segformer/image_processing_segformer.py:99: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(
Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/segformer-b0-finetuned-ade-512-512 and are newly initialized because the shapes did not match:
- decode_head.classifier.weight: found shape torch.Size([150, 256, 1, 1]) in the checkpoint and torch.Size([1, 256, 1, 1]) in the model instantiated
- decode_head.classifier.bias: found shape torch.Size([150]) in the checkpoint and torch.Size([1]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able t

In [32]:
for param in model.segformer.encoder.parameters():
    param.requires_grad = False

In [8]:
image = next(iter(datamodule.train_dataloader()))['image']
print(image.dtype)
print(image.shape)

/home/genrev_kiel_hizon/SAR_flood_segmentation/src/etci_dataset.py:21: RuntimeWarning: divide by zero encountered in divide
  ratio_image = np.clip(np.nan_to_num(vh_image / vv_image, 0), 0, 1)
/home/genrev_kiel_hizon/SAR_flood_segmentation/src/etci_dataset.py:21: RuntimeWarning: invalid value encountered in divide
  ratio_image = np.clip(np.nan_to_num(vh_image / vv_image, 0), 0, 1)


torch.float32
torch.Size([8, 3, 256, 256])


In [8]:
inputs = image_processor(images=image, return_tensors='pt')
inputs['pixel_values'].half()
inputs['pixel_values'].squeeze().shape

torch.Size([8, 3, 512, 512])

In [9]:
inputs['pixel_values'].dtype

torch.float32

In [8]:
model = SegformerForSemanticSegmentation.from_pretrained("nvidia/segformer-b0-finetuned-ade-512-512", ignore_mismatched_sizes=True,
                                                        num_labels=len(id2label), id2label=id2label, label2id=label2id,
                                                        reshape_last_stage=True)

# outputs = model(**inputs)
outputs = model(pixel_values=image)
logits = outputs.logits  # shape (batch_size, num_labels, height/4, width/4)
list(logits.shape)

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/segformer-b0-finetuned-ade-512-512 and are newly initialized because the shapes did not match:
- decode_head.classifier.weight: found shape torch.Size([150, 256, 1, 1]) in the checkpoint and torch.Size([1, 256, 1, 1]) in the model instantiated
- decode_head.classifier.bias: found shape torch.Size([150]) in the checkpoint and torch.Size([1]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[8, 1, 128, 128]

In [161]:
logits[0].shape

torch.Size([1, 64, 64])

In [162]:
sm = torch.nn.Sigmoid()
upsampled_logits = torch.nn.functional.interpolate(logits[0], size=(256,256), mode="bilinear", align_corners=False)
preds=sm(upsampled_logits)
preds = preds.squeeze().detach().numpy()
np.mean(preds)
preds[preds>=0.5] = 1
preds[preds<0.5] = 0
imshow(preds)

ValueError: Input and output must have the same number of spatial dimensions, but got input with spatial dimensions of [64] and output size of (256, 256). Please provide input tensor in (N, C, d1, d2, ...,dK) format and output size in (o1, o2, ...,oK) format.